In [1]:
import hashlib
import tarfile
import os
import zipfile
import requests
import kaggle
import pandas as pd
import torch
from torch import nn
import wandb
from torch.utils import data
from torch.nn import functional as F
#DATA_HUB = dict()
#DATA_URL = 



In [11]:
#下载数据集至指定路径
!kaggle competitions download -c california-house-prices -p /Users/meinv/Downloads/d2l-zh/pytorch/chapter_multilayer-perceptrons/kaggle

 91%|██████████████████████████████████▋   | 27.0M/29.5M [00:02<00:00, 18.4MB/s]
100%|██████████████████████████████████████| 29.5M/29.5M [00:02<00:00, 12.7MB/s]


In [15]:
#切换工作目录
os.chdir('./kaggle')

In [21]:
!unzip california-house-prices.zip

Archive:  california-house-prices.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
#加载数据集
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [5]:
#查看数据集
print(train_data.shape)
print(test_data.shape)

(47439, 41)
(31626, 40)


In [6]:
print(train_data.iloc[0:4, [0, 1, 2, 3, 4,-3, -2, -1]])

   Id            Address  Sold Price  \
0   0        540 Pine Ln   3825000.0   
1   1     1727 W 67th St    505000.0   
2   2     28093 Pine Ave    140000.0   
3   3  10750 Braddock Dr   1775000.0   

                                             Summary          Type  \
0  540 Pine Ln, Los Altos, CA 94022 is a single f...  SingleFamily   
1  HURRY, HURRY.......Great house 3 bed and 2 bat...  SingleFamily   
2  'THE PERFECT CABIN TO FLIP!  Strawberry deligh...  SingleFamily   
3  Rare 2-story Gated 5 bedroom Modern Mediterran...  SingleFamily   

          City    Zip State  
0    Los Altos  94022    CA  
1  Los Angeles  90047    CA  
2   Strawberry  95375    CA  
3  Culver City  90230    CA  


In [7]:
print(test_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

      Id                    Address  \
0  47439        3034 N Coolidge Ave   
1  47440         565 Kenilworth Ave   
2  47441        3028 N Coolidge Ave   
3  47442  3022 N North Coolidge Ave   

                                             Summary          Type  \
0  Live within steps to the scenic views on the L...  SingleFamily   
1                 duplex fixer. Input for comps only  SingleFamily   
2  Live within steps to the scenic views on the L...  SingleFamily   
3  Live within steps to the scenic views on the L...  SingleFamily   

          City    Zip State  
0   Dodgertown  90090    CA  
1  San Leandro  94577    CA  
2  Los Angeles  90039    CA  
3   Dodgertown  90090    CA  


In [8]:
#连接后去除id,price

all_features = pd.concat((train_data.iloc[:,[i for i in range(len(train_data.columns)) if i not in [0, 2, 3]]], test_data.iloc[:, 1:]))


In [9]:
print(all_features.shape)

(79065, 39)


In [10]:
print(all_features.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

             Address          Type  Year built  \
0        540 Pine Ln  SingleFamily      1969.0   
1     1727 W 67th St  SingleFamily      1926.0   
2     28093 Pine Ave  SingleFamily      1958.0   
3  10750 Braddock Dr  SingleFamily      1947.0   

                                        Heating    Zip State Summary  
0  Heating - 2+ Zones, Central Forced Air - Gas  94022    CA     NaN  
1                                   Combination  90047    CA     NaN  
2                                    Forced air  95375    CA     NaN  
3                                       Central  90230    CA     NaN  


In [11]:
#去除summary,city,state
columns_to_remove = ["Summary", "City", "State"]
indices_to_remove = [all_features.columns.get_loc(col) for col in columns_to_remove]
print(indices_to_remove)
all_features = all_features.drop(columns=all_features.columns[indices_to_remove], axis=1)



[38, 35, 37]


In [12]:
# 处理地址
def remove_street_number(address):
    parts = address.split()
    if len(parts) > 1 and parts[0].isdigit():
        parts = parts[1:]
    cleaned_address = ' '.join(parts)

    return cleaned_address

# 在 'Address' 列上应用函数
all_features['Address'] = all_features['Address'].apply(remove_street_number)


In [13]:
#查看处理后的地址

unique_values = all_features["Address"].unique()

print("numbers:", len(unique_values))
#46201还是太多,不要了

numbers: 46201


In [14]:
#看其他列的类别数
unique_counts = all_features.nunique()

print(unique_counts)

Address                        46201
Type                             174
Year built                       169
Heating                         2658
Cooling                          909
Parking                         9911
Lot                            11982
Bedrooms                         277
Bathrooms                         24
Full bathrooms                    17
Total interior livable area     5296
Total spaces                      67
Garage spaces                     57
Region                          1258
Elementary School               3567
Elementary School Score           10
Elementary School Distance       268
Middle School                    808
Middle School Score               10
Middle School Distance           254
High School                      921
High School Score                 10
High School Distance             359
Flooring                        1738
Heating features                1761
Cooling features                 594
Appliances included            11288
L

In [15]:
#去掉其中非数字的并且不同取值数大于500的特征
non_numeric_features = all_features.select_dtypes(exclude=['number'])

unique_counts_non_numeric = non_numeric_features.nunique()

columns_to_remove = unique_counts_non_numeric[unique_counts_non_numeric > 500].index

all_features = all_features.drop(columns = columns_to_remove)

In [16]:
#数据预处理
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [17]:
print(numeric_features.shape)

(18,)


In [18]:
print(numeric_features)

Index(['Year built', 'Lot', 'Bathrooms', 'Full bathrooms',
       'Total interior livable area', 'Total spaces', 'Garage spaces',
       'Elementary School Score', 'Elementary School Distance',
       'Middle School Score', 'Middle School Distance', 'High School Score',
       'High School Distance', 'Tax assessed value', 'Annual tax amount',
       'Listed Price', 'Last Sold Price', 'Zip'],
      dtype='object')


In [19]:
print(all_features.iloc[47439:47442, [0, 1, 2, 3, -3, -2, -1]])

           Type  Year built       Lot Bedrooms  Listed Price  Last Sold Price  \
0  SingleFamily    0.443650 -0.018442        2     -0.155958         0.121116   
1  SingleFamily   -0.300151 -0.017988        3     -0.303132        -0.675027   
2  SingleFamily    0.443650 -0.018442        2     -0.137558         0.000000   

        Zip  
0 -1.471684  
1  0.676922  
2 -1.496106  


In [20]:
# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(79065, 471)

In [21]:
#转换为张量
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values.astype(float), dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values.astype(float), dtype=torch.float32)
train_labels = torch.tensor(train_data['Sold Price'].values.astype(float).reshape(-1, 1), dtype=torch.float32)

In [22]:
print(train_features[0:4,0:4])

tensor([[ 0.0485, -0.0185, -2.0580,  0.0000],
        [-0.2847, -0.0183, -0.3368, -0.1064],
        [-0.0367, -0.0180,  0.5238, -1.1676],
        [-0.1219,  0.0000,  0.5238,  0.9547]])


In [23]:
#先使用简单模型

loss = nn.MSELoss()
in_features = train_features.shape[1]

def get_net1():
    net = nn.Sequential(nn.Linear(in_features,1))
    return net

class  get_net2(nn.Module):
    def __init__(self):
        super().__init__()


def get_net2():
    net = nn.Sequential(nn.Linear(in_features,256),
                        nn.ReLU(),
                        #nn.Dropout(dropout1),
                       nn.Linear(256,128),
                        nn.ReLU(),
                        nn.Linear(128,64),
                        nn.ReLU(),
                        nn.Linear(64,32),
                        nn.ReLU(),
                       # nn.Dropout(dropout2),
                       nn.Linear(32,1))
    return net

In [24]:
#计算误差

def log_rmse(net, features, labels):
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    #log 0会出问题,所以通常限制范围在1到正无穷
    rmse = torch.sqrt(loss(torch.log(clipped_preds),torch.log(labels)))
    return rmse.item()


In [25]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [26]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    wandb.watch(net)
    train_ls, test_ls = [], []
    train_iter = load_array((train_features,train_labels), batch_size)
    optimizer = torch.optim.Adam(net.parameters(),
                                lr = learning_rate,
                                weight_decay = weight_decay)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        record_loss = log_rmse(net.to('cpu'), train_features, train_labels)
        wandb.log({'loss': record_loss,'epoch': epoch})
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, train_labels))
    wandb.finish()
    return train_ls, test_ls



In [27]:
def get_default_device():
    if torch.cuda.is_available():
        return 'cuda'
    elif getattr (torch.backends, 'mps', None) is not None and torch.backends.mps.is_available():
        return 'mps'
    else:
        return 'cpu'

In [28]:
#移动到gpu上

def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    device = get_default_device()
    net.to(device)
    wandb.watch(net)
    train_ls, test_ls = [], []
    train_iter = load_array((train_features, train_labels), batch_size, is_train=True)
    optimizer = torch.optim.Adam(net.parameters(),
                                lr=learning_rate,
                                weight_decay=weight_decay)
    loss_fn = nn.MSELoss()

    for epoch in range(num_epochs):
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            l = loss_fn(net(X), y)
            l.backward()
            optimizer.step()

        record_loss = log_rmse(net.to(device), train_features, train_labels)
        wandb.log({'loss': record_loss, 'epoch': epoch})
        train_ls.append(log_rmse(net, train_features, train_labels))

        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, train_labels))

    wandb.finish()
    return train_ls, test_ls



In [29]:
num_epochs, lr, weight_decay, batch_size, dropout1, dropout2 = 200, 0.001, 0.01, 128, 0.1, 0.5
net = get_net2()


wandb.init(project = "kaggle_california", config={"learning_rate": lr,
                    "weight_decay": weight_decay,
                    "batch_size": batch_size,
                    "total_run": num_epochs,
                    }
          )


In [30]:
device = get_default_device()
train_ls, valid_ls = train(net, train_features.to(device), train_labels.to(device),
                            None, None,
                            num_epochs, lr, weight_decay, batch_size)


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▅▄▄▄▃▂▂▂▂▁▂▁▂▂▁
epoch,199
loss,0.27601


In [262]:
preds = net(test_features).detach().numpy()

test_data['Sold Price'] = pd.Series(preds.reshape(1, -1)[0])
submission = pd.concat([test_data['Id'], test_data['Sold Price']], axis = 1)
submission.to_csv('submission.csv',index=False)

In [263]:
!kaggle competitions submit -c california-house-prices -f submission.csv -m "Message"

100%|█████████████████████████████████████████| 485k/485k [00:02<00:00, 186kB/s]
Successfully submitted to California House Prices